<a href="https://colab.research.google.com/github/edwardly1002/ComputerVision/blob/master/background_subtractor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%cd /content/
!pip install hungarian-algorithm
!pip install scipy==1.5.4

In [2]:
from __future__ import print_function
import cv2 as cv2
from google.colab.patches import cv2_imshow
import numpy as np
from hungarian_algorithm import algorithm as hungarian
from scipy.optimize import linear_sum_assignment as hungary

In [3]:
def cosin_inv(v1, v2):
    dot = v1[0]*v2[0] + v1[1]*v2[1]
    dist1 = np.sqrt(v1[0]*v1[0] + v1[1]*v1[1])
    dist2 = np.sqrt(v2[0]*v2[0] + v2[1]*v2[1])
    cosin = dot/(dist1*dist2)
    if cosin == 0:
        return 999999
    elif dist1==0 or dist2==0:
        return 1
    else:
        cosin = cosin*cosin*cosin*cosin
        return 1/cosin

In [82]:
def get_objects(frame, outputs, counting):
    ###OBJ SUBTRACTOR###
    image = backSub.apply(frame)

    threshold = 10
    image[image != 255] = 0
    # image[image < threshold] = 255

    # image = cv2.dilate(image, None, iterations=2)
    image = cv2.erode(image, None, iterations=2)
    image = cv2.dilate(image, None, iterations=2)
    # image = cv2.dilate(image, None)
    

    contours, _ = cv2.findContours(image, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    image = image.reshape((height, width, 1))
    image = np.concatenate((image, image, image), axis = 2)

    objects = []

    coordinates = []
    for contour in contours:
        (x, y, w, h) = cv2.boundingRect(contour)

        ###WRITE TO COLORED IMAGE###
        if cv2.contourArea(contour) < 100:
            continue
        # cv2.rectangle(frame1, (x, y), (x+w, y+h), (0, 255, 0), 2)
        # cv2.putText(frame1, "Status: {}".format('Movement'), (10, 20), cv2.FONT_HERSHEY_SIMPLEX,
        #             1, (0, 0, 255), 3)
        
        ###WRITE TO GREY IMAGE###
        cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(image, "Status: {}".format('Movement'), (10, 20), cv2.FONT_HERSHEY_SIMPLEX,
                    1, (0, 0, 255), 3)

        ###add points to outputs for personal###
        # outputs[counting].append([x, y, x+w/2, y+h/2, int(np.sqrt(w*h))])

        ###ADD TO OBJECTS LIST###
        objects.append([[x+w/2, y+h/2], -1])
        coordinates.append([x, y])

    return image, outputs, objects, coordinates

In [22]:
|def prepare_configs(input_file):
    cap = cv2.VideoCapture(cv2.samples.findFileOrKeep(input_file))

    width = int( cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height =int( cap.get( cv2.CAP_PROP_FRAME_HEIGHT))

    # fps = int(cap.get(cv2.CAP_PROP_FPS))
    # fps = 10
    fps = 5

    codec = cv2.VideoWriter_fourcc(*"XVID")
    out = cv2.VideoWriter(output_file, codec, fps, (width, height))

    return cap, out, width, height

In [6]:
def numbering_object_1(objs, newObjs, mapping, counting):
    if newObjs == None:
        return newObjs, counting
    elif objs == None or len(objs) == 0:
        for i in range(len(newObjs)):
            counting = counting+1
            newObjs[i][1] = counting
    elif mapping == None:
        return newObjs, counting
    else:
        for i in mapping:
            newObjs[int(i[0][1])-1][1] = objs[(i[0][0])-1][1]
        for i in range(len(newObjs)):
            if newObjs[i][1] == 0:
                counting = counting+1
                newObjs[i][1] = counting

    return newObjs, counting

def numbering_object(objs, newObjs, mapping, counting):
    if newObjs == None:
        return newObjs, counting
    elif objs == None or len(objs) == 0:
        for i in range(len(newObjs)):
            counting = counting+1
            newObjs[i][1] = counting
    elif mapping == None:
        for i in range(len(newObjs)):
            if newObjs[i][1] == -1:
                counting = counting+1
                newObjs[i][1] = counting
    else:
        for i in mapping:
            newObjs[i[1]][1] = objs[i[0]][1]
        for i in range(len(newObjs)):
            if newObjs[i][1] == -1:
                counting = counting+1
                newObjs[i][1] = counting

    return newObjs, counting

In [7]:
###PREPARE INPUT FOR HUNGARIAN ALGO
def create_matrix_1(objs, newObjs): 
    matrix = dict()
    for i in range(len(objs)):
        matrix[i+1] = dict()
        for j in range(len(newObjs)):
            dist = [objs[i][0][0]-newObjs[j][0][0], objs[i][0][1]-newObjs[j][0][1]]
            dist = np.linalg.norm(dist)
            if dist == 0:
                matrix[i+1][str(j+1)] = 999999999
            else:
                dist = 1/dist
                print("DISTANCE = ", dist)
                dist = dist*100000
                # dist = dist*dist
                matrix[i+1][str(j+1)] =  int(dist)
        if len(matrix[i+1]) == 0:
            matrix.pop(i+1)
    return matrix

def create_matrix(objs, newObjs, oldObjs, oldMap): 
    if newObjs == None:
        return []
    matrix = []
    ###EUCLIDIAN DISTANCE###
    for i in range(len(objs)):
        matrix.append([])
        for j in range(len(newObjs)):
            dist = [objs[i][0][0]-newObjs[j][0][0], objs[i][0][1]-newObjs[j][0][1]]
            dist = np.linalg.norm(dist)
            matrix[i].append(dist)

    ###ADD DIRECTION FACTOR###
    if (oldObjs == None) or (oldMap ==None):
        return matrix
    for i in oldMap:
        pOld = oldObjs[i[0]][0]
        p = objs[i[1]][0]
        v1 = [p[0]-pOld[0], p[1]-pOld[1]]
        for j in range(len(newObjs)):
            pNew = newObjs[j][0]
            v2 = [pNew[0]-p[0], pNew[1]-p[1]]
            cosin = cosin_inv(v1,v2)
            matrix[i[1]][j] = matrix[i[1]][j]*cosin
            
    return matrix

In [8]:
def hungarian_algo_1(matrix):
    if len(matrix) < 2:
        return None
    match = hungarian.find_matching(matrix, matching_type="max", return_type="list")
    return match

def hungarian_algo(matrix):
    if len(matrix) < 1:
        return None
    row, col = hungary(matrix)
    match = []
    for i in range(len(row)):
        match.append((row[i], col[i]))
    return match

In [80]:
# !cp "/content/drive/MyDrive/Colab Notebooks/BackgroundSubtract/data/test2.mp4" "/content"
###SETTING INPS###
input_file = "/content/test1.mp4"
algo = "KNN"
output_file = "result1.avi"
output_format = "XVID"
if algo == 'MOG2':
    backSub = cv2.createBackgroundSubtractorMOG2()
elif algo == "KNN":
    backSub = cv2.createBackgroundSubtractorKNN()
else:
    backSub = cv2.bgsegm.createBackgroundSubtractorMOG()

In [83]:
cap, out, width, height = prepare_configs(input_file)

counting = 0
object_num = -1
outputs = [[]]

ret, frame1 = cap.read()
_, outputs, oldObjs, _ = get_objects(frame1, outputs, counting)
oldObjs, object_num = numbering_object(None, oldObjs, None, object_num)

ret, frame2 = cap.read()
_, outputs, objs, _ = get_objects(frame2, outputs, counting)
matrix = create_matrix(oldObjs, objs, None, None)
oldMap = hungarian_algo(matrix)
objs, object_num = numbering_object(oldObjs, objs, oldMap, object_num)


ret, frame3 = cap.read()
while cap.isOpened() and ret:

    #try obj subtractor
    outputs.append([])
    dilated, outputs, newObjs, coordinates = get_objects(frame2, outputs, counting)
    ###FOR OUTING COLORED IMGAGE###
    # image = frame1
    # image.reshape((1080, 1920, 3))

    ###HUNGARY STEP###
    # print("OLD OBJ: ", len(objs), " ============= " , objs)
    # print("NEW OBJ: ", newObjs)
    matrix = create_matrix(objs, newObjs, oldObjs, oldMap)
    # print("MATRIX CREATED: ", matrix)
    mapping = hungarian_algo(matrix)
    # print("MAPPING: ", mapping)
    newObjs, object_num = numbering_object(objs, newObjs, mapping, object_num)
    # print("NEW OBJECT AFTER MANIPULATE", newObjs)

    for i in range(len(newObjs)):
        cv2.putText(dilated, "OBJ: {}".format(newObjs[i][1]), (coordinates[i][0], coordinates[i][1]), cv2.FONT_HERSHEY_SIMPLEX,
                    1, (0, 0, 255), 3)

    oldObjs = objs
    objs = newObjs
    oldMap = mapping

    if counting > 5:
        out.write(dilated)
    else:
        object_num = 0
    frame2 = frame1
    ret, frame1 = cap.read()

    if (counting%50 == 0):
        print("Processing . . . ")
    counting = counting+1

    if cv2.waitKey(40) == 27:
        break

cv2.destroyAllWindows()
cap.release()
out.release()

Processing . . . 


In [ ]:
!ffmpeg -i result.avi output.mp4
from IPython.display import HTML
from base64 import b64encode
mp4 = open('output.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)